# Audio_Visualizer.ipynb

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import IPython.display as ipy
import os
import pathlib
import librosa
from librosa import display as lbdis

In [2]:
# Establish input directory
input_dir = pathlib.Path('songs')
if not input_dir.exists():
    os.mkdir('songs')
    
# Search directory and make libraries
input_content = os.listdir(input_dir)
song_waves = np.empty_like(input_content, dtype=np.ndarray)
song_names = np.empty_like(input_content)
song_box = np.empty_like(input_content, dtype=tuple)

cnt = 0
for i in input_content:
    song_waves[cnt], sr = librosa.load(f'songs/{i}')
    song_names[cnt] = i[:-4]
    song_box[cnt] = (i[:-4], cnt)
    cnt += 1

C:\Users\redem\anaconda3\envs\audiobasics\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
C:\Users\redem\anaconda3\envs\audiobasics\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [4]:
def select_options(Song, Plot, Length):
    if Length == "Full song":
        if Plot == "Wave":
            plot_wave(song_waves[Song])
        elif Plot == "Spectrum":
            plot_spectrum(song_waves[Song])
        elif Plot == "Spectrogram":
            plot_spectrogram(song_waves[Song])           
    else:
        if Length == "30 seconds":
            gen_length = 30 * sr
        if Length == "5 seconds":
            gen_length = 5 * sr
        if Length == "50 milliseconds":
            gen_length = int(0.05 * sr)
            
        start_length = 30 * sr
        if gen_length < len(song_waves[Song]):
            wave_crop = song_waves[Song]
            wave_crop = wave_crop[(int(len(wave_crop)/2)-int(gen_length/2)):(int(len(wave_crop)/2)+int(gen_length/2))]
            if Plot == "Wave":
                plot_wave(wave_crop)
            elif Plot == "Spectrum":
                plot_spectrum(wave_crop)
            elif Plot == "Spectrogram":
                plot_spectrogram(wave_crop) 
        else:
            select_options(Song, Plot, "Full song")
            
def plot_wave(wave):
    plt.figure(figsize=(18,6))
    lbdis.waveplot(wave, alpha=0.5)

def plot_spectrum(wave):
    # derive spectrum using FT
    ft = sp.fft.fft(wave)
    magnitude = np.absolute(ft)
    frequency = np.linspace(0, sr, len(magnitude))

    # plot spectrum
    plt.figure(figsize=(18,6))
    plt.plot(frequency[:(int(len(frequency)/2))], magnitude[:(int(len(magnitude)/2))]) # magnitude spectrum
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude")
    plt.show()

def plot_spectrogram(wave):
    D = librosa.stft(wave)  # STFT of y
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    plt.figure(figsize=(18,6))
    lbdis.specshow(S_db, x_axis='s')
    plt.colorbar()
    
def option_interact(song1, song2, plot1, plot2, length1, length2):
    plot1 = select_options(song1, plot1, length1)
    plot2 = select_options(song2, plot2, length2)
    
    
plot_box = ["Wave", "Spectrum", "Spectrogram"]
length_box = ["Full song", "30 seconds", "5 seconds", "50 milliseconds"]
song2_box = np.array(([""]))
song2_box = np.concatenate((song2_box, song_box))

In [5]:
# build widget ui
song1 = widgets.Dropdown(options=song_box, description="Songs: ")
song2 = widgets.Dropdown(options=song_box)
song_ui = widgets.HBox([song1, song2])
display(song_ui)

plot1 = widgets.Dropdown(options=plot_box, description="Plots: ")
plot2 = widgets.Dropdown(options=plot_box)
plot_ui = widgets.HBox([plot1, plot2])
display(plot_ui)

length1 = widgets.Dropdown(options=length_box, description="Length: ")
length2 = widgets.Dropdown(options=length_box)
length_ui = widgets.HBox([length1, length2])
display(length_ui)

out = widgets.interactive_output(option_interact, {'song1': song1, 'song2': song2, 'plot1': plot1, 'plot2': plot2, 
                                                  'length1': length1, 'length2': length2})
display(out)

C:\Users\redem\anaconda3\envs\audiobasics\lib\site-packages\traitlets\traitlets.py:586: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


Output()